# Assignment 2

Importing Data

In [31]:
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import cho_factor, cho_solve
data = np.load('DATA/ct_data.npz')
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']

In [32]:
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')

X_train shape: (40754, 384)
y_train shape: (40754,)


Question 1a

In [33]:
#defining a function to calculate standard error
def cal_se(data):
    std = np.std(data)
    se = std/ (len(data)**(1/2))
    return se

#verifying the mean on the training set is zero
print(f'The mean on y_train is {round(np.mean(y_train), 8)}.')
print(f'The mean in y_val is {round(np.mean(y_val),8)} +/- {round(cal_se(y_val), 8)}.')

print(f'The mean on the first 5785 entries in y_train is {round(np.mean(y_train[:5785]), 8)} +/- {round(cal_se(y_train[:5785]),8)}.')
#used 8 decimal places as that is the pattern of the data

The mean on y_train is -0.0.
The mean in y_val is -0.21600851 +/- 0.01290338.
The mean on the first 5785 entries in y_train is -0.44247688 +/- 0.01192627.


The standard errors on the mean values of the first 5785 entries of training and the validation dataset suggest that even the edge cases of the mean of the two sets does not meet the overal population mean values. The standard error bars are misleading here because they do not align to what the population mean is??? 

Question 1b

In [34]:
isConstant = []
isDuplicates = np.array(np.zeros(X_train.shape[1]), dtype='bool') 

#identify all the constant columns
for i in range(X_train.shape[1]) :
    isConstant.append((X_train[:,i] == X_train[0][i]).all())

#identify all the columns which are duplicates to previous columns
for i in range(X_train.shape[1]) :
    for k in range(i+1, X_train.shape[1]) :
        if isDuplicates[k] == False:
            isDuplicates[k] = (X_train[:,i] == X_train[:,k]).all()

#create a list with all the columns to remove
columns_to_remove = np.unique(np.hstack((np.nonzero(isDuplicates)[0], np.nonzero(isConstant)[0])))

#remove these columns and name them modified datasets
X_train_mod = np.delete(X_train, columns_to_remove ,1)
X_val_mod = np.delete(X_val, columns_to_remove ,1)
X_test_mod = np.delete(X_test, columns_to_remove ,1)

#check if the shape still aligns to expectations (number of rows stay the same, number of columns are same for all three sets)
print(X_train_mod.shape, X_val_mod.shape, X_test_mod.shape)
# print list of removed columns
print(f'constant columns: {np.nonzero(isConstant)[0]}')
print(f'duplicate columns: {np.nonzero(isDuplicates)[0]}')

(40754, 373) (5785, 373) (6961, 373)
constant columns: [ 59  69 179 189 351]
duplicate columns: [ 69  78  79 179 188 189 199 287 351 359]


Question 2

We want to set up the following matrix equation to solve for the weights and bias of the linear regression model.
$$ \tilde{\Phi}=\left[\begin{matrix} X_{N\times D} & 1_{N\times 1}\\ \sqrt{\alpha}\mathbb{I}_{D\times D} & 0_{D\times 1} \end{matrix}\right],\quad \underline{\tilde{w}}=\left[\begin{matrix} w_{D\times 1}\\ b\end{matrix}\right],\quad \underline{\tilde{y}}=\left[\begin{matrix} y_{D\times 1}\\ 0\end{matrix}\right] $$
$$ E(\underline{w},b)=(\tilde{\Phi}\underline{\tilde{w}}-\underline{\tilde{y}})^T(\tilde{\Phi}\underline{\tilde{w}}-\underline{\tilde{y}}) $$

In [35]:
def fit_linreg(X, yy, alpha):
    # add a column of ones to the X matrix
    Phi = np.hstack((X, np.ones((X.shape[0],1))))
    # add an identity matrix to the Phi matrix for regularization
    # leave the last column as zeros to ignore the bias term
    Phi_til = np.vstack((Phi, np.hstack((np.sqrt(alpha)*np.eye(X.shape[1]), np.zeros((X.shape[1],1))))))
    # compute the new y vector
    yy_til = np.vstack((yy, np.zeros((X.shape[1],1))))
    # compute the weights
    w = np.linalg.lstsq(Phi_til, yy_til, rcond=None)[0]
    return w[:-1], w[-1]

In [36]:
w, b = fit_linreg(X_train_mod, y_train[:, np.newaxis], 30)